In [1]:
import zipfile

import pandas as pd
import numpy as np

from IPython.display import Image, display  #To display images in a notebook
import matplotlib.pyplot as plt
import seaborn as sns

#Data preprocessing
from sklearn.experimental import enable_iterative_imputer   #Required to enable IterativeImputer
from sklearn.impute import IterativeImputer     #to HANDLE missing values
from sklearn.preprocessing import StandardScaler #To standardise variables
from sklearn.preprocessing import LabelEncoder #TO cencode categorical variables

#Splitting Training and test sets
from sklearn.multioutput import MultiOutputRegressor #for mulit output REgression
from sklearn.ensemble import RandomForestRegressor #RandomForest regression model
from xgboost import XGBRegressor    #XGBoost based regression model
from sklearn.feature_selection import SelectFromModel #TO select the most relevant features

#Model performance evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error

from datetime import datetime
import joblib   #To save and load training models
import os

In [2]:
!kaggle datasets download -d excel4soccer/espn-soccer-data


Dataset URL: https://www.kaggle.com/datasets/excel4soccer/espn-soccer-data
License(s): MIT




  0%|          | 0.00/168M [00:00<?, ?B/s]
  1%|          | 1.00M/168M [00:01<03:55, 744kB/s]
  1%|          | 2.00M/168M [00:01<02:13, 1.30MB/s]
  2%|▏         | 3.00M/168M [00:02<01:29, 1.92MB/s]
  2%|▏         | 4.00M/168M [00:02<01:09, 2.46MB/s]
  3%|▎         | 5.00M/168M [00:02<01:00, 2.83MB/s]
  4%|▎         | 6.00M/168M [00:02<00:55, 3.07MB/s]
  4%|▍         | 7.00M/168M [00:03<00:54, 3.11MB/s]
  5%|▍         | 8.00M/168M [00:03<00:50, 3.33MB/s]
  5%|▌         | 9.00M/168M [00:03<00:46, 3.61MB/s]
  6%|▌         | 10.0M/168M [00:03<00:46, 3.58MB/s]
  7%|▋         | 11.0M/168M [00:04<00:44, 3.69MB/s]
  7%|▋         | 12.0M/168M [00:04<00:44, 3.70MB/s]
  8%|▊         | 13.0M/168M [00:04<00:41, 3.90MB/s]
  8%|▊         | 14.0M/168M [00:04<00:40, 3.94MB/s]
  9%|▉         | 15.0M/168M [00:05<00:43, 3.70MB/s]
 10%|▉         | 16.0M/168M [00:05<00:47, 3.37MB/s]
 10%|█         | 17.0M/168M [00:05<00:45, 3.44MB/s]
 11%|█         | 18.0M/168M [00:06<00:43, 3.61MB/s]
 11%|█▏        | 19.0

In [3]:
import zipfile

zip_path = "espn-soccer-data.zip"
extract_path = "espn_data"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed.")


Extraction completed.
